In [2]:
import torch
import torchvision
import onnx
import torch.nn as nn
import torch.nn.functional as F

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        print("after 1 conv:", x.size())  # [1, 6, 28, 28]

        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        print("after 1 conv:", x.size())  # [1, 6, 28, 28]

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # num_flat_features：计算张量x的总特征量（把每个数字都看出是一个特征，即特征总量）
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [8]:
net = Net()
params = list(net.parameters())
print(len(params))
print(type(params[0]))
for i in range(10):
    print("第{}个参数形状为{}".format(i,params[i].size()))
input = torch.randn(1, 1, 32, 32)
print(input)
print('conv1输出的形状：',net.conv1(input).size()) # conv1输出的形状

out = net(input)
print('输出层形状',out.size())
print('输出层结果：',out)

10
<class 'torch.nn.parameter.Parameter'>
第0个参数形状为torch.Size([6, 1, 5, 5])
第1个参数形状为torch.Size([6])
第2个参数形状为torch.Size([16, 6, 5, 5])
第3个参数形状为torch.Size([16])
第4个参数形状为torch.Size([120, 400])
第5个参数形状为torch.Size([120])
第6个参数形状为torch.Size([84, 120])
第7个参数形状为torch.Size([84])
第8个参数形状为torch.Size([10, 84])
第9个参数形状为torch.Size([10])
tensor([[[[-0.0946, -0.4331,  0.1049,  ..., -1.7064, -0.4263,  0.1183],
          [-2.1867, -0.0615,  1.0661,  ..., -1.1888,  0.2416,  1.0151],
          [-1.0415,  0.1843, -0.1180,  ...,  0.9071, -0.6675, -0.6326],
          ...,
          [ 0.6196, -2.0734,  0.0409,  ..., -0.5851, -0.8084, -0.9392],
          [ 2.9923, -0.8401, -2.0703,  ..., -0.9866, -2.2073, -0.2497],
          [-0.1664,  0.5513, -0.3112,  ...,  0.8007, -0.6792,  0.8023]]]])
conv1输出的形状： torch.Size([1, 6, 28, 28])
after 1 conv: torch.Size([1, 6, 14, 14])
after 1 conv: torch.Size([1, 16, 5, 5])
输出层形状 torch.Size([1, 10])
输出层结果： tensor([[-0.0773, -0.0849,  0.0981, -0.1587,  0.0063, -0.0250,  0.0210,  

In [4]:
x = torch.ones(1,1,32,32)
net = Net()
net.load_state_dict(torch.load('net_params.pth'))
prediction = net(x)
print(prediction)

RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "conv2.weight", "conv2.bias". 
	Unexpected key(s) in state_dict: "net.0.weight", "net.0.bias", "net.1.weight", "net.1.bias", "net.1.running_mean", "net.1.running_var", "net.1.num_batches_tracked", "net.4.weight", "net.4.bias", "net.5.weight", "net.5.bias", "net.5.running_mean", "net.5.running_var", "net.5.num_batches_tracked". 